In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt
    

In [23]:
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [20]:
def get_data(driver):
    iteration = True
    time.sleep(20)
    #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
    #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
    # 더보기 클릭
    driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()
    
    time.sleep(5)
    # 숙박 날짜 가져오기
    dates = driver.find_elements_by_class_name("_34Xs-BQm")
    stay_dates = []
    for d in dates:
        stay_date = convert_date(d.text)
        stay_dates.append(stay_date)
        if stay_date < datetime.date(2019,7,1):
            iteration = False
    
    # 닉네임 가져오기
    nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
    # 별점 가져오기
    rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
    scores = []
    for rc in rating_codes:
        cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
        score = int(cls_attr[1])
        scores.append(score)

    # 리뷰 가져오기
    reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")

        
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    
    for i in range(len(nicknames)):
        df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
        #print(df.loc[i])
    return df, iteration

In [25]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test.csv', encoding="utf-8-sig")
    
    return data

In [24]:
url = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d12707683-Reviews-Novotel_Ambassador_Seoul_Yongsan-Seoul.html"
data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지


,nickname,score,review,stay_date
0,james,30,부대시설 과 식사는 괜찮습니다.\n객실이 문제예요. 화장실 환풍기 없고 위에 객실과...,2021-06-01
1,odeuny,50,어린이날을 맞아 가족과 함께 방문한 노보텔 용산!\n체크인시 해프닝도 있었지만 친절...,2021-05-01
2,수연 김,40,노보텔에 묵을때도 묵지않을때도 메가바이트커피를 좋아해서 자주갑니다.메가버거도 좋아하...,2021-02-01
3,Eunhyeok lee,50,전번 리뷰에서는 맘에안드는점이 너무 많아서 혹평을 쏟아놨었는데요\n이번에 크리스마스...,2020-12-01
4,georjina,50,36시간 패키지로 알차게 잘 놀다 왔어요!\n침대 머리 위에 콘센트가 있어서 편했어...,2020-07-01
5,dani,50,태풍 때문에 휴가가 어그러져 호캉스라도 즐기고자 왔어요.\n내부 시설을 이용하지 못...,2020-09-01
6,Funderbar,10,플래티넘 회원으로 룸은 업그레이드 받았습니다. 거기까지만 무난\n\n체크아웃도 하기...,2020-06-01
7,Eunhyeok lee,20,기존에 박당 25만원 내고 머물렀을때보다\n11만원 주고 머문 이번이 만족도가 더 ...,2020-04-01
8,Mike,50,"직원분들이 정말정말 친절했습니다.\n용산역과 연결되어 전철 이용이 편리하고,\n아이...",2020-01-01
9,Chan lee,50,아주 깔끔하고 전망좋은 곳에서 쉬다가 왔습니다. 용산이라는 위치가 장거리 연애하는 ...,2019-08-01


In [15]:
data_sample.to_csv('./hotel_review_novotel_yongsan2.csv', encoding="utf-8-sig")

In [26]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    data = data[data['stay_date'] > datetime.date(2019,7,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [27]:
preprocessing_data(data_sample)

,nickname,score,review,stay_date
0,james,30,부대시설 과 식사는 괜찮습니다. 객실이 문제예요. 화장실 환풍기 없고 위에 객실과 ...,2021-06-01
1,odeuny,50,어린이날을 맞아 가족과 함께 방문한 노보텔 용산! 체크인시 해프닝도 있었지만 친절하...,2021-05-01
2,수연 김,40,노보텔에 묵을때도 묵지않을때도 메가바이트커피를 좋아해서 자주갑니다.메가버거도 좋아하...,2021-02-01
3,Eunhyeok lee,50,전번 리뷰에서는 맘에안드는점이 너무 많아서 혹평을 쏟아놨었는데요 이번에 크리스마스 ...,2020-12-01
4,dani,50,태풍 때문에 휴가가 어그러져 호캉스라도 즐기고자 왔어요. 내부 시설을 이용하지 못하...,2020-09-01
5,georjina,50,36시간 패키지로 알차게 잘 놀다 왔어요! 침대 머리 위에 콘센트가 있어서 편했어요...,2020-07-01
6,Funderbar,10,플래티넘 회원으로 룸은 업그레이드 받았습니다. 거기까지만 무난 체크아웃도 하기 전...,2020-06-01
7,Eunhyeok lee,20,기존에 박당 25만원 내고 머물렀을때보다 11만원 주고 머문 이번이 만족도가 더 낮...,2020-04-01
8,Mike,50,"직원분들이 정말정말 친절했습니다. 용산역과 연결되어 전철 이용이 편리하고, 아이파크...",2020-01-01
9,Eunhyeok lee,30,일반룸 장점 보통호텔보다 훨씬 넓음 어메니티 노보텔 공정무역 쓰레기 아니고 옥시젠...,2019-10-01


In [30]:
tmp = preprocessing_data(data_sample)
tmp['review2'] = tmp['review'].apply(lambda X : X.encode('utf-8', 'ignore').decode('utf-8'))
tmp

<ipython-input-30-2d152371ce3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['review2'] = tmp['review'].apply(lambda X : X.encode('utf-8', 'ignore').decode('utf-8'))


,nickname,score,review,stay_date,review2
0,james,30,부대시설 과 식사는 괜찮습니다. 객실이 문제예요. 화장실 환풍기 없고 위에 객실과 ...,2021-06-01,부대시설 과 식사는 괜찮습니다. 객실이 문제예요. 화장실 환풍기 없고 위에 객실과 ...
1,odeuny,50,어린이날을 맞아 가족과 함께 방문한 노보텔 용산! 체크인시 해프닝도 있었지만 친절하...,2021-05-01,어린이날을 맞아 가족과 함께 방문한 노보텔 용산! 체크인시 해프닝도 있었지만 친절하...
2,수연 김,40,노보텔에 묵을때도 묵지않을때도 메가바이트커피를 좋아해서 자주갑니다.메가버거도 좋아하...,2021-02-01,노보텔에 묵을때도 묵지않을때도 메가바이트커피를 좋아해서 자주갑니다.메가버거도 좋아하...
3,Eunhyeok lee,50,전번 리뷰에서는 맘에안드는점이 너무 많아서 혹평을 쏟아놨었는데요 이번에 크리스마스 ...,2020-12-01,전번 리뷰에서는 맘에안드는점이 너무 많아서 혹평을 쏟아놨었는데요 이번에 크리스마스 ...
4,dani,50,태풍 때문에 휴가가 어그러져 호캉스라도 즐기고자 왔어요. 내부 시설을 이용하지 못하...,2020-09-01,태풍 때문에 휴가가 어그러져 호캉스라도 즐기고자 왔어요. 내부 시설을 이용하지 못하...
5,georjina,50,36시간 패키지로 알차게 잘 놀다 왔어요! 침대 머리 위에 콘센트가 있어서 편했어요...,2020-07-01,36시간 패키지로 알차게 잘 놀다 왔어요! 침대 머리 위에 콘센트가 있어서 편했어요...
6,Funderbar,10,플래티넘 회원으로 룸은 업그레이드 받았습니다. 거기까지만 무난 체크아웃도 하기 전...,2020-06-01,플래티넘 회원으로 룸은 업그레이드 받았습니다. 거기까지만 무난 체크아웃도 하기 전...
7,Eunhyeok lee,20,기존에 박당 25만원 내고 머물렀을때보다 11만원 주고 머문 이번이 만족도가 더 낮...,2020-04-01,기존에 박당 25만원 내고 머물렀을때보다 11만원 주고 머문 이번이 만족도가 더 낮...
8,Mike,50,"직원분들이 정말정말 친절했습니다. 용산역과 연결되어 전철 이용이 편리하고, 아이파크...",2020-01-01,"직원분들이 정말정말 친절했습니다. 용산역과 연결되어 전철 이용이 편리하고, 아이파크..."
9,Hammmeee,50,부모님 결혼30주년 기념으로 가족끼리 호캉스 다녀왔어요~! 언니가 결혼해서 언니네식...,2019-10-01,부모님 결혼30주년 기념으로 가족끼리 호캉스 다녀왔어요~! 언니가 결혼해서 언니네식...


In [146]:
def to_csv():
    data.to_csv('./hotel_review_mondrian_itaewon.csv', encoding="utf-8-sig")

In [36]:
str = '안녕하세요❌ ㅋㅋ'
str.encode('euc-kr', 'ignore').decode('euc-kr') 

'안녕하세요 ㅋㅋ'